In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf

SEED = 2018

np.random.seed(SEED)
tf.set_random_seed(SEED)

from tqdm import tqdm
tqdm.pandas()

import os
print(os.listdir("../input"))

['embeddings', 'sample_submission.csv', 'test.csv', 'train.csv']
['embeddings', 'sample_submission.csv', 'test.csv', 'train.csv']


In [2]:
# ## some config values 
# embed_size = 300 # how big is each word vector
# #max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
# max_features = 120000
# maxlen = 72 # max number of words in a question to use

## some config values 
embed_size = 600 # how big is each word vector
#max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
max_features = None
maxlen = 57 # max number of words in a question to use

In [3]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, Lambda
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, BatchNormalization
from keras.optimizers import Adam, Nadam
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback

from keras import initializers, regularizers, constraints, optimizers, layers

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from keras.engine.topology import Layer
# from tensorflow.keras.layers import Layer

import gc, re
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, auc, precision_recall_curve

Using TensorFlow backend.


In [4]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"}

mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

In [5]:
def load_and_prec():
    train_df = pd.read_csv("../input/train.csv")
    test_df = pd.read_csv("../input/test.csv")
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
       # Clean the text
    train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_text(x))
    test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: clean_text(x))
    
    # Clean numbers
    train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: clean_numbers(x))
    test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: clean_numbers(x))
    
    # Clean speelings
    train_df["question_text"] = train_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
    test_df["question_text"] = test_df["question_text"].progress_apply(lambda x: replace_typical_misspell(x))
    
    train_X = train_df["question_text"].fillna("_##_").values
    splits = list(StratifiedKFold(n_splits=10,random_state=2018).split(train_X,train_df['target'].values))
    ## fill up the missing values
    test_X = test_df["question_text"].fillna("_##_").values

    ## split to train and val
    train_X, val_X = train_X[splits[0][0]],train_X[splits[0][1]]
    
    ## Get the target values
#     train_y = train_df['target'].values
    train_y = train_df['target'].values[splits[0][0]]
    val_y = train_df['target'].values[splits[0][1]]
    
    ## Tokenize the sentences
#     tokenizer = Tokenizer(num_words=max_features)
    tokenizer = Tokenizer(num_words=max_features, filters='', lower=True)
    tokenizer.fit_on_texts(list(train_X)+list(test_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    val_X = tokenizer.texts_to_sequences(val_X)
    test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    val_X = pad_sequences(val_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    #shuffling the data
    np.random.seed(SEED)
    trn_idx = np.random.permutation(len(train_X))
    val_idx = np.random.permutation(len(val_X))

    train_X = train_X[trn_idx]
    val_X = val_X[val_idx]
    train_y = train_y[trn_idx]
    val_y = val_y[val_idx]    
    
    return train_X, val_X, test_X, train_y, val_y, tokenizer.word_index
#     return train_X, test_X, train_y, tokenizer.word_index

In [6]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'

    emb_mean,emb_std = -0.005838499,0.48782197
    embed_size = 300

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    #nb_words = len(word_index)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8") as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= nb_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: break
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'

    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = 300

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    #nb_words = len(word_index)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8", errors='ignore') as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= nb_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [7]:
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    rocauc = roc_auc_score(y_true, y_proba)
    p, r, _ = precision_recall_curve(y_true, y_proba)
    prauc = auc(r, p)
    search_result = {'threshold': best_threshold, 'f1': best_score, 'rocauc': rocauc, 'prauc': prauc}
    return search_result

In [8]:
# https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
            scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [9]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)
        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [10]:
def model_gru_conv_3(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x0 = Bidirectional(CuDNNLSTM(128, kernel_initializer=initializers.glorot_uniform(seed = 2018), return_sequences=True))(x)
    x1 = Bidirectional(CuDNNGRU(64, kernel_initializer=initializers.glorot_uniform(seed = 2018), return_sequences=True))(x0)
    z = Conv1D(64, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=2018), activation = "tanh")(x1)
    y1 = GlobalMaxPooling1D()(x1)
    y2 = GlobalMaxPooling1D()(z)
    x = concatenate([y1,y2])
    #x = Dropout(0.1)(x)
    #x = Dense(8, kernel_initializer=initializers.he_uniform(seed=2018), activation='relu')(x)
#     x = Dense(32, activation="relu")(x)
#     x = Dropout(0.1)(x)
#     x = Dense(1, activation="sigmoid")(x)
    outp= Dense(1, kernel_initializer=initializers.he_uniform(seed=2018), activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
#     model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy'])
    return model

In [11]:
SEED=2018
def model_RCNN(embedding_matrix, hidden_dim_1=128, hidden_dim_2=64,max_features=max_features):
    embedding_matrix = np.concatenate([embedding_matrix,np.zeros((1,np.shape(embedding_matrix)[1]))])
    print(np.shape(embedding_matrix))
    
    left_context = Input(shape=(maxlen,))
    document = Input(shape=(maxlen,))
    right_context = Input(shape=(maxlen,))

    embedder = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)
    doc_embedding = embedder(document)
    doc_embedding = SpatialDropout1D(0.2)(doc_embedding)
    l_embedding = embedder(left_context)
    l_embedding = SpatialDropout1D(0.2)(l_embedding)
    r_embedding = embedder(right_context)
    r_embedding = SpatialDropout1D(0.2)(r_embedding)
    
    # I use LSTM RNNs instead of vanilla RNNs as described in the paper.
    forward = CuDNNLSTM(hidden_dim_1, return_sequences = True)(l_embedding) # See equation (1).
    backward = CuDNNLSTM(hidden_dim_1, return_sequences = True)(r_embedding) # See equation (2).
    # Keras returns the output sequences in reverse order.
    backward = Lambda(lambda x: K.reverse(x, axes = 1))(backward)
    together = concatenate([forward, doc_embedding, backward], axis = 2) # See equation (3).

#     semantic = TimeDistributed(Dense(hidden_dim_2, kernel_initializer=initializers.he_uniform(seed=SEED), activation = "relu"))(together)
#     semantic = Bidirectional(CuDNNGRU(128, kernel_initializer=initializers.glorot_uniform(seed = SEED), return_sequences=True))(together)
    semantic = Conv1D(hidden_dim_2*8, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=SEED), activation = "tanh")(together) # See equation (4).

    # Keras provides its own max-pooling layers, but they cannot handle variable length input
    # (as far as I can tell). As a result, I define my own max-pooling layer here.
    pool_rnn = GlobalMaxPool1D()(semantic) # See equation (5).

    output = Dense(1, kernel_initializer=initializers.he_uniform(seed=SEED), activation = "sigmoid")(pool_rnn) # See equations (6) and (7).

    model = Model(inputs = [left_context, document, right_context], outputs = output)
    model.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = ["accuracy"])
#     model.compile(optimizer = Nadam(), loss = 'binary_crossentropy', metrics = ["accuracy"])
    return model

In [12]:
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x0 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
#     x1 = Bidirectional(CuDNNGRU(64, return_sequences=True))(x0)
    x2 = Bidirectional(CuDNNGRU(96, return_sequences=True))(x0)
#     x2 = CuDNNGRU(64, return_sequences=True)(x1)
    y2 = GlobalMaxPooling1D()(x2)
#     x = Concatenate()([y2, y1])
#     y2 = BatchNormalization()(y2)
    y2 = Dropout(0.1)(y2)
    x = Dense(1, activation="sigmoid")(y2)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
#     model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy'])
    
    return model

In [13]:
def model_lstm_max(embedding_matrix):
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x0 = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
#     x1 = Bidirectional(CuDNNGRU(128, return_sequences=True))(x0)
#     x2 = Bidirectional(CuDNNGRU(64, return_sequences=True))(x0)
    x1 = Bidirectional(CuDNNGRU(64, kernel_initializer=initializers.glorot_uniform(seed = 2018), return_sequences=True))(x0)
    z = Conv1D(64, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=2018), activation = "tanh")(x1)
#     x2 = CuDNNGRU(64, return_sequences=True)(x1)
#     y1 = Attention(maxlen)(x1)
    y2 = Attention(maxlen)(z)
#     x = Concatenate()([y2, y1])
#     x = Dense(64, activation="relu")(x)
#     x = Dense(16, activation="relu")(y2)
#     x = Dropout(0.1)(x)
#     x = Dense(1, activation="sigmoid")(x)
    x = Dense(1, activation="sigmoid")(y2)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
#     model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy'])
    
    return model

In [14]:
def get_train_list(train_X):
    return [np.concatenate((np.ones((np.shape(train_X)[0],1))*max_features+1,train_X[:,1:]),1),train_X,np.concatenate((np.ones((np.shape(train_X)[0],1))*max_features+1,train_X[:,::-1][:,1:]),1)]

# https://www.kaggle.com/strideradu/word2vec-and-gensim-go-go-go
def RCNN_train_pred(model, epochs=2):
    train_X_list = get_train_list(train_X)
    test_X_list=get_train_list(test_X)
    val_X_list=get_train_list(val_X)
    for e in range(epochs):
        model.fit(train_X_list, train_y, batch_size=512, epochs=1, validation_data=(val_X_list, val_y),callbacks=[clr])
    pred_val_y = model.predict(val_X_list, batch_size=1024, verbose=0)
    pred_test_y = model.predict(test_X_list, batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y

In [15]:
def train_pred(model, epochs=2):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y),verbose=1,callbacks=[clr])

    pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y, pred_test_y

In [16]:
train_X, val_X, test_X, train_y, val_y,  word_index = load_and_prec()
max_features = len(word_index)
print(max_features)
embedding_matrix_1 = load_glove(word_index)
embedding_matrix_2 = load_fasttext(word_index)
# embedding_matrix_3 = load_para(word_index)
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
embedding_matrix = np.concatenate([embedding_matrix_1, embedding_matrix_2], axis = 1)
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_3], axis = 0)
np.shape(embedding_matrix)

  1%|          | 9706/1306122 [00:00<00:13, 97056.62it/s]

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


100%|██████████| 375806/375806 [00:03<00:00, 120036.25it/s]


206308


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


(206308, 600)

In [17]:
outputs = []

In [18]:
clr = CyclicLR(base_lr=0.001, max_lr=0.003,step_size=300., mode='exp_range', gamma=0.99994)

In [19]:
pred_val_y, pred_test_y = RCNN_train_pred(model_RCNN(embedding_matrix, hidden_dim_1=128, hidden_dim_2=64,max_features=max_features+1), epochs = 5)
outputs.append([pred_val_y, pred_test_y, 'RCNN'])
results = threshold_search(val_y, pred_val_y)
print(results)
print(confusion_matrix(val_y,pred_val_y>results['threshold']))

(206309, 600)
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 437s 371us/step - loss: 0.1089 - acc: 0.9572 - val_loss: 0.0996 - val_acc: 0.9609
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 434s 369us/step - loss: 0.0962 - acc: 0.9614 - val_loss: 0.0965 - val_acc: 0.9613
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
 705024/1175509 [================>.............] - ETA: 2:47 - loss: 0.0894 - acc: 0.9638

In [20]:
clr = CyclicLR(base_lr=0.001, max_lr=0.003,step_size=300., mode='exp_range', gamma=0.99994)

In [21]:
pred_val_y, pred_test_y = train_pred(model_lstm_atten(embedding_matrix), epochs = 4)
outputs.append([pred_val_y, pred_test_y, 'LSTM w/ max'])
results = threshold_search(val_y, pred_val_y)
print(results)
print(confusion_matrix(val_y,pred_val_y>results['threshold']))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 383s 326us/step - loss: 0.1124 - acc: 0.9555 - val_loss: 0.1013 - val_acc: 0.9603
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 382s 325us/step - loss: 0.0980 - acc: 0.9610 - val_loss: 0.0973 - val_acc: 0.9614
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 382s 325us/step - loss: 0.0915 - acc: 0.9632 - val_loss: 0.0963 - val_acc: 0.9614
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 382s 325us/step - loss: 0.0858 - acc: 0.9653 - val_loss: 0.0950 - val_acc: 0.9622
{'threshold': 0.37, 'f1': 0.6963921431485691, 'rocauc': 0.9696106381518341, 'prauc': 0.7277014855292742}
[[119430   3102]
 [  2107   5974]]


In [22]:
clr = CyclicLR(base_lr=0.001, max_lr=0.003,step_size=300., mode='exp_range', gamma=0.99994)

In [23]:
pred_val_y, pred_test_y = train_pred(model_gru_conv_3(embedding_matrix), epochs = 4)
outputs.append([pred_val_y, pred_test_y, 'LSTM conv 3'])
results = threshold_search(val_y, pred_val_y)
print(results)
print(confusion_matrix(val_y,pred_val_y>results['threshold']))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 368s 313us/step - loss: 0.1104 - acc: 0.9565 - val_loss: 0.0992 - val_acc: 0.9604
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 367s 312us/step - loss: 0.0968 - acc: 0.9612 - val_loss: 0.0966 - val_acc: 0.9615
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 366s 312us/step - loss: 0.0898 - acc: 0.9637 - val_loss: 0.0973 - val_acc: 0.9617
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 366s 312us/step - loss: 0.0843 - acc: 0.9658 - val_loss: 0.0954 - val_acc: 0.9620


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'threshold': 0.41000000000000003, 'f1': 0.6947890818858562, 'rocauc': 0.9697070255710357, 'prauc': 0.7260976900356666}
[[119567   2965]
 [  2201   5880]]


In [24]:
clr = CyclicLR(base_lr=0.001, max_lr=0.003,step_size=300., mode='exp_range', gamma=0.99994)

In [25]:
pred_val_y, pred_test_y = train_pred(model_lstm_max(embedding_matrix), epochs = 4)
outputs.append([pred_val_y, pred_test_y, 'LSTM w/ atten'])
results = threshold_search(val_y, pred_val_y)
print(results)
print(confusion_matrix(val_y,pred_val_y>results['threshold']))

Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 365s 310us/step - loss: 0.1137 - acc: 0.9550 - val_loss: 0.1025 - val_acc: 0.9590
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 363s 309us/step - loss: 0.0999 - acc: 0.9602 - val_loss: 0.0989 - val_acc: 0.9605
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 363s 309us/step - loss: 0.0942 - acc: 0.9622 - val_loss: 0.0984 - val_acc: 0.9607
Train on 1175509 samples, validate on 130613 samples
Epoch 1/1
1175509/1175509 [==============================] - 363s 309us/step - loss: 0.0887 - acc: 0.9642 - val_loss: 0.0976 - val_acc: 0.9609


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'threshold': 0.28, 'f1': 0.6878483194484343, 'rocauc': 0.9682803001857361, 'prauc': 0.7192713958578798}
[[119194   3338]
 [  2095   5986]]


In [26]:
# from sklearn.linear_model import LogisticRegression
train_meta_data=np.concatenate([np.reshape(i[0],(-1,1)) for i in outputs],axis=1)
test_meta_data = np.concatenate([np.reshape(i[1],(-1,1)) for i in outputs],axis=1)
print('Corr in Train:')
print(pd.DataFrame(train_meta_data).corr())
print('Corr in Test')
print(pd.DataFrame(test_meta_data).corr())

Corr in Train:
          0         1         2         3
0  1.000000  0.946259  0.946565  0.935086
1  0.946259  1.000000  0.957998  0.949041
2  0.946565  0.957998  1.000000  0.943401
3  0.935086  0.949041  0.943401  1.000000
Corr in Test
          0         1         2         3
0  1.000000  0.944884  0.946100  0.935549
1  0.944884  1.000000  0.957167  0.949515
2  0.946100  0.957167  1.000000  0.943870
3  0.935549  0.949515  0.943870  1.000000


In [27]:
coefs = [0.35,0.25,0.2,0.2]
pred_val_y = np.sum([outputs[i][0]*coefs[i] for i in range(len(outputs))], axis = 0)
results = threshold_search(val_y, pred_val_y)
print(results)
print(confusion_matrix(val_y,pred_val_y>results['threshold']))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'threshold': 0.34, 'f1': 0.7067257256090455, 'rocauc': 0.9726642457438482, 'prauc': 0.7439581563969572}
[[119518   3014]
 [  2018   6063]]


In [28]:
# pred_test_y = np.sum([outputs[i][1] * reg.coef_[i] for i in range(len(outputs))], axis = 0)
coefs = [0.35,0.25,0.2,0.2]
# pred_test_y = np.mean([outputs[i][1] for i in range(len(outputs))], axis = 0)
pred_test_y = np.sum([outputs[i][1]*coefs[i] for i in range(len(coefs))], axis = 0)

pred_test_y = (pred_test_y > results['threshold']).astype(int)
test_df = pd.read_csv("../input/test.csv", usecols=["qid"])
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)